<a href="https://colab.research.google.com/github/Prajwal-Rathod/RAG_1/blob/main/QAbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qU \
    langchain-pinecone==0.1.1 \
    langchain-openai==0.1.8 \
    langchain-text-splitters==0.2.0 \
    langchain==0.2.1 \
    pinecone-notebooks==0.1.1

In [ ]:
import os

# initialize connection to pinecone (orget API key at app.pinecone.io)
if not os.environ.get("PINCONE_API_Key"):
    from pinecone_notebooks.colab import Authenticate
    Authenticate()

In [ ]:
api_key = os.environ.get("Pincone_API_Key_after_authentication") or "Pincone_API_Key_after_authentication"

# available at platform.openai.com/api-keys
os.environ['open_ai_api_key'] = os.environ.get('open_ai_api_key') or "open_ai_api_key"

In [ ]:
from langchain_text_splitters import MarkdownHeaderTextSplitter

# Define your markdown document with information about Quantum Solutions Inc.
markdown_document = """
## Introduction

Welcome to the world of Quantum Solutions Inc., a leader in innovative technology and business solutions. Our company is dedicated to providing cutting-edge services that revolutionize industries and drive growth. Whether you're a seasoned entrepreneur or just starting, Quantum Solutions Inc. has the tools and expertise to help you succeed. Let's explore the incredible range of services, setup process, and troubleshooting tips for this innovative business.

## Company Overview

Quantum Solutions Inc. offers a variety of services designed to meet the diverse needs of modern businesses:

- **Quantum Analytics Platform**: Our state-of-the-art analytics platform uses quantum computing principles to deliver unparalleled insights and predictive analytics. This allows businesses to make data-driven decisions with unprecedented accuracy.

- **Hyper-Scale Cloud Solutions**: We provide scalable cloud solutions that grow with your business. Our infrastructure is designed to handle the most demanding workloads, ensuring your applications run smoothly and efficiently.

- **Next-Gen Cybersecurity**: Protect your business with our advanced cybersecurity solutions. We use cutting-edge technology to safeguard your data against the latest threats, ensuring your operations remain secure.

- **AI-Driven Marketing Tools**: Our AI-driven marketing tools help you reach your target audience more effectively. By leveraging machine learning, we create personalized marketing campaigns that deliver measurable results.

- **Blockchain Integration Services**: We offer comprehensive blockchain integration services that help you implement secure and transparent business processes. From supply chain management to smart contracts, we have the expertise to guide you through the blockchain revolution.

## Use Cases

Quantum Solutions Inc. services can be applied across various industries to drive innovation and growth. Here are some hypothetical scenarios where our solutions could be utilized:

- **Data-Driven Decision Making**: Use our Quantum Analytics Platform to gain insights into market trends, customer behavior, and operational efficiency. This enables businesses to make informed decisions that drive success.

- **Scalable Infrastructure**: Implement our Hyper-Scale Cloud Solutions to ensure your business infrastructure can handle increasing demands. This is ideal for growing startups and established enterprises alike.

- **Enhanced Security**: Protect sensitive information with our Next-Gen Cybersecurity solutions. This is crucial for industries such as finance, healthcare, and e-commerce, where data security is paramount.

- **Effective Marketing**: Leverage our AI-Driven Marketing Tools to create targeted campaigns that resonate with your audience. This helps businesses maximize their marketing ROI and grow their customer base.

- **Blockchain Transparency**: Use our Blockchain Integration Services to streamline business processes and increase transparency. This is particularly beneficial for industries like logistics, where tracking and verifying goods is essential.

## Getting Started

Setting up your services with Quantum Solutions Inc. is straightforward and efficient. Follow these steps to unlock the full potential of our offerings:

1. **Contact Us**: Reach out to our sales team to discuss your business needs and find the best solutions for you. We'll provide a detailed overview of our services and how they can benefit your business.

2. **Onboarding Process**: Once you've selected your services, our onboarding team will guide you through the setup process. This includes configuring your analytics platform, setting up cloud infrastructure, and integrating cybersecurity measures.

3. **Training and Support**: We offer comprehensive training to ensure your team can effectively use our tools and platforms. Additionally, our support team is available 24/7 to assist with any issues or questions.

4. **Implementation**: Our technical experts will assist with the implementation of your chosen solutions. This includes deploying cloud services, integrating AI-driven tools, and setting up blockchain systems.

5. **Optimization**: We continually monitor and optimize your services to ensure they deliver maximum value. Our team will work with you to make adjustments as needed and ensure your business remains at the forefront of innovation.

## Troubleshooting

Even the most advanced solutions can encounter issues. Here are some common problems and their solutions:

- **Issue: Analytics Platform Not Loading**

    - **Solution**: Ensure your internet connection is stable. Clear your browser cache and try accessing the platform again. If the issue persists, contact our support team for assistance.

- **Issue: Slow Cloud Performance**

    - **Solution**: Check for any scheduled maintenance or outages on our status page. If none are reported, try scaling up your resources through the cloud management dashboard. For further help, reach out to our support team.

- **Issue: Security Alerts**

    - **Solution**: Review the security logs to identify the source of the alert. Ensure all software is up to date and run a full security scan. If the issue remains unresolved, contact our cybersecurity experts.

- **Issue: Marketing Tool Malfunction**

    - **Solution**: Verify that all inputs and configurations are correct. Restart the tool and attempt your task again. If problems continue, our support team is ready to help.

- **Issue: Blockchain Integration Error**

    - **Solution**: Ensure all nodes are correctly configured and communicating. Check for any software updates or patches that may need to be applied. For persistent issues, our blockchain specialists can provide assistance.
"""

# Define the headers to split on
headers_to_split_on = [
    ("##", "Header 2")
]

# Create the MarkdownHeaderTextSplitter instance
markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on, strip_headers=False
)

# Split the markdown document
md_header_splits = markdown_splitter.split_text(markdown_document)

# Print the resulting splits
for idx, split in enumerate(md_header_splits):
    print(f"Section {idx + 1}:\n{split}\n")


Section 1:
page_content="## Introduction  \nWelcome to the world of Quantum Solutions Inc., a leader in innovative technology and business solutions. Our company is dedicated to providing cutting-edge services that revolutionize industries and drive growth. Whether you're a seasoned entrepreneur or just starting, Quantum Solutions Inc. has the tools and expertise to help you succeed. Let's explore the incredible range of services, setup process, and troubleshooting tips for this innovative business." metadata={'Header 2': 'Introduction'}

Section 2:
page_content='## Company Overview  \nQuantum Solutions Inc. offers a variety of services designed to meet the diverse needs of modern businesses:  \n- **Quantum Analytics Platform**: Our state-of-the-art analytics platform uses quantum computing principles to deliver unparalleled insights and predictive analytics. This allows businesses to make data-driven decisions with unprecedented accuracy.  \n- **Hyper-Scale Cloud Solutions**: We provi

In [ ]:
from langchain_openai import OpenAIEmbeddings

model_name = 'text-embedding-3-small'
embeddings = OpenAIEmbeddings(
    model=model_name,
    openai_api_key=os.environ.get('open_ai_api_key')
)

In [ ]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=os.environ.get("Pincone_API_Key_after_authentication"))

In [ ]:
from pinecone import ServerlessSpec

cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'

spec = ServerlessSpec(cloud=cloud, region=region)

In [ ]:
index_name = "rag-getting-started"

In [ ]:
import time

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=1536,
        metric="cosine",
        spec=spec
    )
    # wait for index to be ready
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)

In [ ]:
from langchain_pinecone import PineconeVectorStore

namespace = "Quantum Solutions Inc"

docsearch = PineconeVectorStore.from_documents(
    documents=md_header_splits,
    index_name=index_name,
    embedding=embeddings,
    namespace=namespace
)

time.sleep(1)

In [ ]:
index = pc.Index(index_name)

for ids in index.list(namespace=namespace):
    query = index.query(
        id=ids[0],
        namespace=namespace,
        top_k=1,
        include_values=True,
        include_metadata=True
    )
    print(query)

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA

llm = ChatOpenAI(
    openai_api_key=os.environ.get('open_ai_api_key'),
    model_name='gpt-3.5-turbo',
    temperature=0.0
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever()
)

In [ ]:
query1 = "What are the first 3 steps for getting started with Quantum Solutions Inc. services?"

query2 = "My AI-Driven Marketing Tool is malfunctioning. What should I do?"


In [ ]:
query1_with_knowledge = qa.invoke(query1)
query1_without_knowledge = llm.invoke(query1)

print(query1_with_knowledge)
print()
print(query1_without_knowledge)

In [ ]:
query2_with_knowledge = qa.invoke(query2)
query2_without_knowledge = llm.invoke(query2)

print(query2_with_knowledge)
print()
print(query2_without_knowledge)